In [2]:
import pandas as pd

In [3]:
df = pd.read_pickle('help_cleaned2.pk')

In [4]:
df_filtered = df[(df['tokenLength'] > 200) & (df['tokenLength'] < 600)].head(2)
df_filtered.shape

(2, 4)

In [5]:
from transformers import AutoTokenizer, AutoModel

# Initialize the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("Open-Orca/Mistral-7B-OpenOrca")
model = AutoModel.from_pretrained("Open-Orca/Mistral-7B-OpenOrca")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [53]:

embeddings = []


In [21]:
import torch

# Generate embeddings
for text in df_filtered['cleanText']:
    inputs = tokenizer(text, return_tensors="pt")

    # Get embeddings
    with torch.no_grad():
        outputs = model(**inputs)

    # The embeddings
    currentEmbedding = outputs.last_hidden_state
    embeddings.append(currentEmbedding)

In [63]:
# Generate embeddings
for text in df_filtered['cleanText']:
    # Tokenize and truncate/pad text to a fixed max length
    inputs = tokenizer(text, return_tensors="pt")

    # Get embeddings
    with torch.no_grad():
        output = model(**inputs)
        # Get the embeddings of the last layer
        last_layer_embeddings = output.last_hidden_state
        # Optionally, you might want to take the mean of embeddings across all tokens
        mean_embeddings = torch.mean(last_layer_embeddings, dim=1)
        embeddings.append(mean_embeddings.numpy())


In [64]:
len(embeddings)

4

In [65]:
embeddings_backup = embeddings

In [66]:
embeddings = embeddings_backup

In [67]:
embeddings

[tensor([ -1.8227,   0.3743,  -1.7445,  ...,   1.3818, -10.5403,   3.2852]),
 tensor([ -1.8227,   0.3743,  -1.7445,  ...,  -3.3136, -11.6187,   1.8194]),
 array([[ 1.4418099 ,  1.5221655 , -0.70869404, ..., -2.1970756 ,
         -5.3257027 ,  0.7526725 ]], dtype=float32),
 array([[ 1.7773298 ,  0.5177772 ,  0.09558902, ..., -2.6588442 ,
         -4.2135096 , -0.9662012 ]], dtype=float32)]

In [34]:
embeddings

[tensor([[[ -1.8227,   0.3743,  -1.7445,  ...,   0.1591,  -2.0071,   2.7506],
          [ -4.7891,  -1.8972,  13.1883,  ...,   7.5025,  -5.1783,  -7.6219],
          [  3.4071,   1.4723,   3.9228,  ...,  -4.1923,  -2.6856,   5.2247],
          ...,
          [  4.3639,   4.0947,  -6.7094,  ...,   4.9330, -14.5754,  10.4948],
          [  8.5122,   2.7841,   0.5844,  ...,   2.1795,  -9.3484,  10.2494],
          [ -2.6860,  -0.9758,  -2.8921,  ...,   1.3818, -10.5403,   3.2852]]]),
 tensor([[[ -1.8227,   0.3743,  -1.7445,  ...,   0.1591,  -2.0071,   2.7506],
          [ -4.6855,  -1.7113,   9.0106,  ...,  -6.4281,   3.1038,   1.5737],
          [  3.0037,  -4.0166,   8.5847,  ...,  -3.4914,  -3.5845,  -7.7997],
          ...,
          [  7.9818,   6.9891,  -4.5732,  ...,   7.2540, -15.4666,   4.5123],
          [ 10.9016,   4.6319,   2.9511,  ...,   3.1043,  -7.9114,   5.9313],
          [  1.3389,  -4.8215,  -2.8140,  ...,  -3.3136, -11.6187,   1.8194]]])]

In [61]:
# Flatten embeddings since they're in shape (1, seq_length, emb_dim)
embeddings = [emb.reshape(-1) for emb in embeddings]

# FAISS requires the data to be in np.float32
embeddings = np.array(embeddings).astype('float32')

# Create a FAISS index
d = embeddings.shape[1]  # dimension of embeddings
index = faiss.IndexFlatL2(d)

# Add embeddings to the index
index.add(embeddings)

# Save the index to a file
faiss.write_index(index, "faiss_index.bin")

ValueError: only one element tensors can be converted to Python scalars

In [ ]:

# 1. Create Sample Texts
sample_texts = ["Hello world", "Hello there", "i want to fuck you","How are you world","you look haat"]


# Function to generate embeddings
def get_embeddings(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return torch.mean(outputs.last_hidden_state, dim=1).numpy()

# Generate embeddings for sample texts
embeddings = np.vstack([get_embeddings(text) for text in sample_texts])

# 3. Store Embeddings in a FAISS Index
d = embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(embeddings.astype('float32'))


In [79]:
index.ntotal

5

In [81]:

# 4. Create a Test String
test_text = "usual text in any method in computer program"

# 5. Generate Embedding for the Test String
test_embedding = get_embeddings(test_text)

# 6. Search Through the FAISS Index
k = 5  # number of nearest neighbors
D, I = index.search(test_embedding.astype('float32'), k)

# Print the nearest neighbors
print("Test Text: ", test_text)
for i, idx in enumerate(I[0]):
    print(f"Rank {i+1}: (Score: {D[0][i]}) {sample_texts[idx]}")


Test Text:  usual text in any method in computer program
Rank 1: (Score: 51009.28125) i want to fuck you
Rank 2: (Score: 52647.2421875) How are you world
Rank 3: (Score: 69148.03125) Hello world
Rank 4: (Score: 74820.953125) you look haat
Rank 5: (Score: 80765.234375) Hello there


In [85]:
tokenizer

LlamaTokenizerFast(name_or_path='Open-Orca/Mistral-7B-OpenOrca', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '<|im_end|>', 'unk_token': '<unk>', 'pad_token': '[PAD]'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=True, lstrip=True, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=True, lstrip=True, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	32000: AddedToken("<|im_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	32001: AddedToken("<|im_start|>", rstrip=True, lstrip=True, single_word=False, normalized=False, special=True),
	32002: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),

In [99]:
import torch
import numpy as np
import faiss
from transformers import AutoTokenizer, AutoModel
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
# 1. Create Sample Texts
sample_texts = ['''
The TypeError: float() argument must be a string or a number, not 'BaseModelOutputWithPast' suggests that there's an attempt to convert a model output object directly to a float, which is not possible. This error typically occurs when working with models from Hugging Face's Transformers library, where the model's output is a complex object containing various elements like the hidden states, attentions, etc., rather than a single numeric value.
The TypeError: float() argument must be a string or a number, not 'BaseModelOutputWithPast' suggests that there's an attempt to convert a model output object directly to a float, which is not possible. This error typically occurs when working with models from Hugging Face's Transformers library, where the model's output is a complex object containing various elements like the hidden states, attentions, etc., rather than a single numeric value.
The TypeError: float() argument must be a string or a number, not 'BaseModelOutputWithPast' suggests that there's an attempt to convert a model output object directly to a float, which is not possible. This error typically occurs when working with models from Hugging Face's Transformers library, where the model's output is a complex object containing various elements like the hidden states, attentions, etc., rather than a single numeric value.
The TypeError: float() argument must be a string or a number, not 'BaseModelOutputWithPast' suggests that there's an attempt to convert a model output object directly to a float, which is not possible. This error typically occurs when working with models from Hugging Face's Transformers library, where the model's output is a complex object containing various elements like the hidden states, attentions, etc., rather than a single numeric value.
The TypeError: float() argument must be a string or a number, not 'BaseModelOutputWithPast' suggests that there's an attempt to convert a model output object directly to a float, which is not possible. This error typically occurs when working with models from Hugging Face's Transformers library, where the model's output is a complex object containing various elements like the hidden states, attentions, etc., rather than a single numeric value.
The TypeError: float() argument must be a string or a number, not 'BaseModelOutputWithPast' suggests that there's an attempt to convert a model output object directly to a float, which is not possible. This error typically occurs when working with models from Hugging Face's Transformers library, where the model's output is a complex object containing various elements like the hidden states, attentions, etc., rather than a single numeric value.
''', "Hello there", "i want to sex you","How are you world","you look haat"]

# Function to generate embeddings
def get_embeddings(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding='max_length')
    with torch.no_grad():
        outputs = model(**inputs)
    return torch.mean(outputs.last_hidden_state, dim=1).numpy()

# Generate embeddings for sample texts
embeddings = np.vstack([get_embeddings(text) for text in sample_texts])

# 3. Store Embeddings in a FAISS Index
d = embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(embeddings.astype('float32'))

# 4. Create a Test String
test_text = "usual text in any method in computer program"
# test_text = "im want to get laid"

# 5. Generate Embedding for the Test String
test_embedding = get_embeddings(test_text)

# 6. Search Through the FAISS Index
k = 5  # number of nearest neighbors
D, I = index.search(test_embedding.astype('float32'), k)

# Print the nearest neighbors
print("Test Text: ", test_text)
for i, idx in enumerate(I[0]):
    print(f"Rank {i+1}: (Score: {D[0][i]}) {sample_texts[idx]}")


KeyboardInterrupt: 

In [101]:
from sklearn.neighbors import NearestNeighbors
# 1. Create Sample Texts
sample_texts = ['''
The TypeError: float() argument must be a string or a number, not 'BaseModelOutputWithPast' suggests that there's an attempt to convert a model output object directly to a float, which is not possible. This error typically occurs when working with models from Hugging Face's Transformers library, where the model's output is a complex object containing various elements like the hidden states, attentions, etc., rather than a single numeric value.
The TypeError: float() argument must be a string or a number, not 'BaseModelOutputWithPast' suggests that there's an attempt to convert a model output object directly to a float, which is not possible. This error typically occurs when working with models from Hugging Face's Transformers library, where the model's output is a complex object containing various elements like the hidden states, attentions, etc., rather than a single numeric value.
The TypeError: float() argument must be a string or a number, not 'BaseModelOutputWithPast' suggests that there's an attempt to convert a model output object directly to a float, which is not possible. This error typically occurs when working with models from Hugging Face's Transformers library, where the model's output is a complex object containing various elements like the hidden states, attentions, etc., rather than a single numeric value.
The TypeError: float() argument must be a string or a number, not 'BaseModelOutputWithPast' suggests that there's an attempt to convert a model output object directly to a float, which is not possible. This error typically occurs when working with models from Hugging Face's Transformers library, where the model's output is a complex object containing various elements like the hidden states, attentions, etc., rather than a single numeric value.
The TypeError: float() argument must be a string or a number, not 'BaseModelOutputWithPast' suggests that there's an attempt to convert a model output object directly to a float, which is not possible. This error typically occurs when working with models from Hugging Face's Transformers library, where the model's output is a complex object containing various elements like the hidden states, attentions, etc., rather than a single numeric value.
The TypeError: float() argument must be a string or a number, not 'BaseModelOutputWithPast' suggests that there's an attempt to convert a model output object directly to a float, which is not possible. This error typically occurs when working with models from Hugging Face's Transformers library, where the model's output is a complex object containing various elements like the hidden states, attentions, etc., rather than a single numeric value.
''', "Hello there", "i want to sex you","How are you world","you look haat"]

# Function to generate embeddings
def get_embeddings(text):
    inputs = tokenizer(text, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    return torch.mean(outputs.last_hidden_state, dim=1).numpy()

# Generate embeddings for sample texts
embeddings = np.vstack([get_embeddings(text) for text in sample_texts])

# 3. Use Scikit-learn's Nearest Neighbors
nbrs = NearestNeighbors(n_neighbors=5, algorithm='ball_tree').fit(embeddings)

# 4. Create a Test String
test_text = "usual text in any method in computer program"
# test_text = "im want to get laid"

# 5. Generate Embedding for the Test String
test_embedding = get_embeddings(test_text)

# 6. Search Through the Nearest Neighbors
distances, indices = nbrs.kneighbors(test_embedding)

# Print the nearest neighbors
print("Test Text: ", test_text)
for i, idx in enumerate(indices[0]):
    print(f"Rank {i+1}: (Distance: {distances[0][i]}) {sample_texts[idx]}")


Test Text:  usual text in any method in computer program
Rank 1: (Distance: 224.914500851701) i want to sex you
Rank 2: (Distance: 229.44989313903582) How are you world
Rank 3: (Distance: 256.23700617157806) 
The TypeError: float() argument must be a string or a number, not 'BaseModelOutputWithPast' suggests that there's an attempt to convert a model output object directly to a float, which is not possible. This error typically occurs when working with models from Hugging Face's Transformers library, where the model's output is a complex object containing various elements like the hidden states, attentions, etc., rather than a single numeric value.
The TypeError: float() argument must be a string or a number, not 'BaseModelOutputWithPast' suggests that there's an attempt to convert a model output object directly to a float, which is not possible. This error typically occurs when working with models from Hugging Face's Transformers library, where the model's output is a complex object c

In [102]:
pip install txtai

     |████████████████████████████████| 198 kB 6.8 MB/s eta 0:00:01
     |████████████████████████████████| 776 kB 80.9 MB/s eta 0:00:01
     |████████████████████████████████| 17.6 MB 37.8 MB/s eta 0:00:0100:01
  Attempting uninstall: regex
    Found existing installation: regex 2021.4.4
    Uninstalling regex-2021.4.4:
      Successfully uninstalled regex-2021.4.4
Note: you may need to restart the kernel to use updated packages.


In [103]:
from txtai.embeddings import Embeddings

# Create embeddings model, backed by sentence-transformers & transformers
embeddings = Embeddings(path="sentence-transformers/nli-mpnet-base-v2")

data = [
  "US tops 5 million confirmed virus cases",
  "Canada's last fully intact ice shelf has suddenly collapsed, " +
  "forming a Manhattan-sized iceberg",
  "Beijing mobilises invasion craft along coast as Taiwan tensions escalate",
  "The National Park Service warns against sacrificing slower friends " +
  "in a bear attack",
  "Maine man wins $1M from $25 lottery ticket",
  "Make huge profits without work, earn up to $100,000 a day"
]

# Index the list of text
embeddings.index(data)

print(f"{'Query':20} Best Match")
print("-" * 50)

# Run an embeddings search for each query
for query in ("feel good story", "climate change", "public health story", "war",
              "wildlife", "asia", "lucky", "dishonest junk"):
    # Extract uid of first result
    # search result format: (uid, score)
    uid = embeddings.search(query, 1)[0][0]

    # Print text
    print(f"{query:20} {data[uid]}")

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/587 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Query                Best Match
--------------------------------------------------
feel good story      Maine man wins $1M from $25 lottery ticket
climate change       Canada's last fully intact ice shelf has suddenly collapsed, forming a Manhattan-sized iceberg
public health story  US tops 5 million confirmed virus cases
war                  Beijing mobilises invasion craft along coast as Taiwan tensions escalate
wildlife             The National Park Service warns against sacrificing slower friends in a bear attack
asia                 Beijing mobilises invasion craft along coast as Taiwan tensions escalate
lucky                Maine man wins $1M from $25 lottery ticket
dishonest junk       Make huge profits without work, earn up to $100,000 a day


In [ ]:
from txtai.embeddings import Embeddings

# Create embeddings model, backed by sentence-transformers & transformers
# embeddings = Embeddings(path="mistralai/Mistral-7B-v0.1")
embeddings = Embeddings({"path": "mistralai/Mistral-7B-v0.1", "device": "cpu"})


In [2]:


data = [
  "US tops 5 million confirmed virus cases",
  "Canada's last fully intact ice shelf has suddenly collapsed, " +
  "forming a Manhattan-sized iceberg",
  "Beijing mobilises invasion craft along coast as Taiwan tensions escalate",
  "The National Park Service warns against sacrificing slower friends " +
  "in a bear attack",
  "Maine man wins $1M from $25 lottery ticket",
  "Make huge profits without work, earn up to $100,000 a day"
]

# Index the list of text
embeddings.encode(data)
# results = embeddings.encode([text])

print(f"{'Query':20} Best Match")
print("-" * 50)

# Run an embeddings search for each query
for query in ("feel good story", "climate change", "public health story", "war",
              "wildlife", "asia", "lucky", "dishonest junk"):
    # Extract uid of first result
    # search result format: (uid, score)
    uid = embeddings.search(query, 1)[0][0]

    # Print text
    print(f"{query:20} {data[uid]}")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 500.00 MiB. GPU 0 has a total capacty of 10.75 GiB of which 3.81 MiB is free. Process 46125 has 10.73 GiB memory in use. Of the allocated memory 10.18 GiB is allocated by PyTorch, and 1.65 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF